In [1]:
# Adaptive histogram equalization
import cv2
import os, ntpath, re, shutil
from PIL import Image, ImageOps

In [2]:
DATA_DIRECTORY = os.path.join('..', '..', 'data')

ORIGINAL_DATA = os.path.join(DATA_DIRECTORY, 'original', 'asl-alphabet')
FABRICATED_DATA = os.path.join(DATA_DIRECTORY, 'fabricated', 'asl_alphabet')


FABRICATED_TRAIN_DIR = os.path.join(FABRICATED_DATA, 'split_asl_alphabet_train')
FABRICATED_TEST_DIR = os.path.join(FABRICATED_DATA, 'split_asl_alphabet_test')

NEW_TRAIN_DIR = os.path.join(FABRICATED_DATA, 'processed_asl_train')

AHE_TEST_DIR = os.path.join(FABRICATED_DATA, 'processed_ahe_asl_test')
AHE_NEG_TEST_DIR = os.path.join(FABRICATED_DATA, 'processed_ahe_neg_asl_test')
NEW_UNEDITED_TEST_DIR = os.path.join(FABRICATED_DATA, 'processed_original_test')

# Used to quickly create all of these directories
FABRICATED_DIRS = [
    NEW_TRAIN_DIR,
    AHE_TEST_DIR,
    AHE_NEG_TEST_DIR,
    NEW_UNEDITED_TEST_DIR
]

# Used to declaritively define relatinships of where we will be saving images
DIRECTORY_MAPPINGS = [
    # for the training directory, we just put all of the processed/original images in there
    {'orig_dirname': FABRICATED_TRAIN_DIR, 'new_dirnames': {'ahe': NEW_TRAIN_DIR, 'ahe_inverted': NEW_TRAIN_DIR, 'original': NEW_TRAIN_DIR}},
    # We actually will create a new directory for each time of preprocessing for test images
    {'orig_dirname': FABRICATED_TEST_DIR, 'new_dirnames': {'ahe': AHE_TEST_DIR, 'ahe_inverted': AHE_NEG_TEST_DIR, 'original': NEW_UNEDITED_TEST_DIR}}
]

# A, B, C, D, DELETE, etc
ASL_FOLDERS = os.listdir(FABRICATED_TRAIN_DIR)

In [3]:
TEST_RUN = True
DELETE_FOLDERS = True # Delete/Empty folders before starting

In [4]:
def delete_folders():
    for _dir in FABRICATED_DIRS:
        try:
            shutil.rmtree(_dir)
        except:
            # nothing to do here - it's likely that the directories just don't exist yet
            return

In [ ]:
def create_folders_with_letter(letter):
    for _dir in FABRICATED_DIRS:
        dir_with_letter = os.path.join(_dir, letter)
        os.makedirs(dir_with_letter)

In [ ]:
def create_folders():
    # create the output folders matching the train data set folders
    successfully_created = 0
    unsuccessfully_created = 0

    for sign_name in ASL_FOLDERS:   
        try:
            create_folders_with_letter(sign_name)
            successfully_created = successfully_created + 1
        except:
            unsuccessfully_created = unsuccessfully_created + 1


    print('successfully created {} sets of folders'.format(successfully_created))
    print('unsuccessfully created {} sets of folders (probably already existed)'.format(unsuccessfully_created))

In [ ]:
def create_filename_for_fabricated_image(original_filename, enhancement):
    parts = re.split('\.', original_filename)
    file_body = '.'.join(parts[:-1])
    filename_extension = parts[-1]
    # the sticky part between the filename and the filename extension
    glue = enhancement
    return '{}_{}.{}'.format(file_body, glue, filename_extension)
create_filename_for_fabricated_image('A.jpg', 'reg')

'A_reg.jpg'

In [ ]:
def create_ahe_image(img):
    # create a CLAHE object (Arguments are optional).
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
    ahe_img = clahe.apply(img)
    return Image.fromarray(ahe_img)

In [ ]:
def create_inverse_image(img):
    return ImageOps.invert(img)

In [ ]:
def process_and_save_each_image_in_path(in_dir, sign_name, dir_mapping):
    for infile in os.listdir(in_dir):
        original_img = cv2.imread(os.path.join(in_dir, infile), 0)
        ahe_img = create_ahe_image(original_img)
        invert_ahe_img = create_inverse_image(ahe_img)

        imgs_to_save = [Image.fromarray(original_img), ahe_img, invert_ahe_img]
        enhancements = ['original', 'ahe', 'ahe_inverted']
        for img, enhancement in zip(imgs_to_save, enhancements):
            outdir_base = dir_mapping[enhancement]
            out_dir = os.path.join(outdir_base, sign_name)
            outfile_name = create_filename_for_fabricated_image(infile, enhancement)
            outfile_path = os.path.join(out_dir, outfile_name)

            img.save(outfile_path)
        if TEST_RUN:
            break

In [ ]:
# create enhanced images using adaptive histogram equalization (ahe)
# create negative images of the ahe images
def enhance_and_save_images():
    if DELETE_FOLDERS:
        delete_folders()
    create_folders()

    # directory_settings Will be done for training and test image directories
    for directory_settings in DIRECTORY_MAPPINGS:
        from_dir = directory_settings['orig_dirname']
        print('copying from {}'.format(from_dir))
        dir_mapping = directory_settings['new_dirnames']
        for dirs in dir_mapping:
            print('-- copying to {}'.format(dir_mapping[dirs]))

        for sign_name in ASL_FOLDERS:
            in_dir = os.path.join(from_dir, sign_name)

            process_and_save_each_image_in_path(in_dir, sign_name, dir_mapping)
enhance_and_save_images()

successfully created 29 sets of folders
unsuccessfully created 0 sets of folders (probably already existed)
copying from ../../data/fabricated/asl_alphabet/split_asl_alphabet_train
-- copying to ../../data/fabricated/asl_alphabet/processed_asl_train
-- copying to ../../data/fabricated/asl_alphabet/processed_asl_train
-- copying to ../../data/fabricated/asl_alphabet/processed_asl_train
